In [ ]:
# 1) Import Libraries

import pandas as pd
import os
import requests
import io
import numpy as np
import datetime
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# 2) Import Data
# 2.1) Set Folder Path

PATH_A = 'C:\\Users\\AgaPakos\\Documents\\UE\\ERASMUS\\Bolonia\\STUDY\\Machine Learning Systems for Data Science - Systems and Algorithms for Data Science\\covid19\\python_files'
PATH_M = 'C:\\Users\\maxim\\OneDrive\\Desktop\\Unibo\\Machine Learning Systems for Data Science_90477\\Module II\\Final Project'
PATH_D = ''

In [ ]:
# 2.2) Import Air Quality data from Local Path

# Select one of the paths here
air_PATH = os.path.join(PATH_A, 'new-york-air-quality.csv')
air = pd.read_csv(air_PATH, parse_dates = ['date'])
print(air.columns) # renaming neccesary for the last 4 columns
print(air)
air.info()
air.rename(columns={' pm25': 'pm25', ' o3': 'o3', ' no2': 'no2', ' co': 'co'}, inplace = True)
print(air)
# Overview
print(air.head())
air.info() # We can see that the columns in air need to be converted to 'int64'
air.describe(include = 'all', datetime_is_numeric = True) # 'datetime_is_numeric = True' is used to silence a known Python warning
print(air.head())

In [ ]:
# 2.3) Import NYC Health data from Github repository

url_health = 'https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv'
download_health = requests.get(url_health).content
health = pd.read_csv(io.StringIO(download_health.decode('utf-8')), sep = ',', parse_dates= ['date_of_interest'])

# Overview
# print(health.head())
health.info()
health.describe(include = 'all', datetime_is_numeric = True) # 'datetime_is_numeric = True' is used to silence a known Python warning

In [ ]:
# 3) Process and merge data
# 3.1) Process data
# 3.1.1) Change datetime columns from air and health to date-format without loosing the Dtype 'datetime64[ns]
air['date'] = pd.to_datetime(air['date'])
health['date_of_interest'] = pd.to_datetime(health['date_of_interest'])

# 3.1.2) Sort air and health by their date columns
air = air.sort_values(by = 'date')
health = health.sort_values(by = 'date_of_interest')

# 3.1.3) Check the date ranges
air_date_range_min = min(air['date'].dt.strftime('%Y-%m-%d'))
air_date_range_max = max(air['date'].dt.strftime('%Y-%m-%d'))
print(f'Date range for air: {air_date_range_min}, {air_date_range_max}')
health_date_range_min = min(health['date_of_interest'].dt.strftime('%Y-%m-%d'))
health_date_range_max = max(health['date_of_interest'].dt.strftime('%Y-%m-%d'))
print(f'Date range for health: {health_date_range_min}, {health_date_range_max}')
# We can see that no date range includes the other. We therefore need an outer merge in 3.2)

In [ ]:
# 3.1.4) Find missing date rows in both air and health and add the missing rows

# 3.1.4.1) air
# Find the missing date rows
air = air.set_index('date') # Set date as index
air.index = pd.to_datetime(air.index) # format to date time object
air_date_missing = pd.date_range(start = str(air_date_range_min), end = str(air_date_range_max)).difference(air.index)
print(air)
print(air_date_missing) 
print("For air: " + str(air_date_missing.size) + " date rows are missing (before fix)")
# We can see that air is missing rows.

# Add the missing date rows
air_date_range = pd.date_range(str(air_date_range_min), str(air_date_range_max)) # creates perfect date range
air.index = pd.DatetimeIndex(air.index)
air = air.reindex(air_date_range)
air_date_missing = pd.date_range(start = str(air_date_range_min), end = str(air_date_range_max)).difference(air.index)
print("For air: " + str(air_date_missing.size) + " date rows are missing (after fix)")
# Missing rows have been added.

air.reset_index(inplace = True, drop = False) # needed to undo the index setting above; inplace = True means the 
# original air dataframe is targeted; drop = False retrieves the original date column.
air.rename(columns={'index': 'date'}, inplace = True) # returning to original column name, so this cell can be re-run easier

# 3.1.4.2) health
# Find the missing date rows
health = health.set_index('date_of_interest') # Set date value as index
health.index = pd.to_datetime(health.index) # format to date time object
health_date_missing = pd.date_range(start = str(health_date_range_min), end = str(health_date_range_max)).difference(health.index)
print(health_date_missing)
print("For health: " + str(health_date_missing.size) + " date rows are missing (before fix)")
# We can see that health is not missing any rows.

# Add the missing date rows
health_date_range = pd.date_range(str(health_date_range_min), str(health_date_range_max)) # creates perfect date range
health.index = pd.DatetimeIndex(health.index)
health = health.reindex(health_date_range)
health_date_missing = pd.date_range(start = str(health_date_range_min), end = str(health_date_range_max)).difference(health.index)
print("For health: " + str(health_date_missing.size) + " date rows are missing (after fix)")

health.reset_index(inplace = True, drop = False) # needed to undo the index setting above; inplace = True means the 
# original halth dataframe is targeted; drop = False retrieves the original date column.
health.rename(columns={'index': 'date_of_interest'}, inplace = True) # returning to original column name, so this cell can be re-run easier

In [ ]:
# 3.2) Merge data from air and health into the final dataframe; process the data
# 3.2.1) Outer merge air and health by their date columns; change all variables of interest to Dtype 'int64'
df_merged = pd.merge(air, health, how = 'outer', left_on = 'date', right_on = 'date_of_interest')
print(df_merged.iloc[0, [0, 5]]) # to check that the merged dataframe starts from the first possible date of both air and health
print(df_merged.iloc[-1, [0, 5]]) # to check that the merged dataframe ends on the last possible date of both air and health
print(f'Date range for air: {air_date_range_min}, {air_date_range_max}')
print(f'Date range for health: {health_date_range_min}, {health_date_range_max}')
# The date range matches.

# 3.2.2) Create date_merged column which combines the entries from both date columns
df_merged['date_merged'] = df_merged['date']
df_merged['date_merged'] = df_merged.date_merged.combine_first(df_merged.date_of_interest)

# 3.2.3 Create final dataframe
set_start_date = '2020-03-03'
set_end_date = '2020-06-26'
index_start_date = df_merged.loc[df_merged.date_merged == set_start_date].index[0]
index_end_date = df_merged.loc[df_merged.date_merged == set_end_date].index[0]
df_final = df_merged.copy()
df_final = df_final.loc[index_start_date:index_end_date, ['date_merged', 'pm25', 'no2', 'DEATH_COUNT']]
df_final.head()

In [ ]:
# 3.3 Process final dataframe

# 3.3.1) Rename columns
df_final.rename(columns={'date_merged': 'date', 'DEATH_COUNT': 'death'}, inplace = True)
df_final

# 3.3.2) Change datatypes; for pm25 and no2 we replace empty values by NaN and use a linear interpolation to replace NaN
df_final = df_final.replace(r'^\s*$', np.nan, regex=True) 
df_final[['pm25', 'no2']] = df_final[['pm25', 'no2']].astype(float).interpolate(method = 'linear').round(0).astype(np.int64)
df_final.loc[:, 'death'] = df_final.loc[:, 'death'].astype(np.int64)

# 3.3.3) Create additional log and difference variables
df_final[['ln_pm25', 'ln_no2', 'ln_death']] = np.log(df_final[['pm25', 'no2', 'death']]) # warning: log(0) = -Inf
df_final[['d_pm25', 'd_no2', 'd_death']] = df_final[['pm25', 'no2', 'death']].diff(periods = 1, axis = 0)
df_final.replace([np.inf, -np.inf], np.nan, inplace = True) # replace Inf and -Inf by NaN


In [ ]:
# 3.4) View the whole final dataframe
with pd.option_context('display.max_rows', None):
   print(df_final)

df_final.head()
df_final.info()

# 3.5) Descriptive plots
## 3.5.1) Scatterplots


### a) Deaths vs PM2.5

In [ ]:
df_final.plot.scatter(x='pm25',
                     y='death',
                     ylim=(-10,610)
                     )
# Add title and axis names
plt.title('Deaths vs PM2.5 scatter chart')
plt.xlabel('PM2.5')
plt.ylabel('Deaths')

#Create linear regression
X = df_final.iloc[:, 1].values.reshape(-1, 1)  # values converts it into a numpy array ->PM2.5
Y = df_final.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column-> Death
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions
plt.plot(X, Y_pred, color='red')

plt.show()


### b) Deaths vs NO2

In [ ]:
df_final.plot.scatter(x='no2',
                     y='death',
                     ylim=(-10,600))
# Add title and axis names
plt.title('Deaths vs NO2 scatter chart')
plt.xlabel('N02')
plt.ylabel('Deaths')

#Create linear regression
X = df_final.iloc[:, 2].values.reshape(-1, 1)  # values converts it into a numpy array -> NO2
Y = df_final.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column-> Death
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions
plt.plot(X, Y_pred, color='red')

plt.show()


### c) Deaths vs lnPM2.5 

In [ ]:
df_final.plot.scatter(x='ln_pm25',
                     y='death',
                     ylim=(-10,600))
# Add title and axis names
plt.title('Deaths vs lnPM2.5 scatter chart')
plt.xlabel('lnPM2.5')
plt.ylabel('Deaths')

#Create linear regression
X = df_final.iloc[:, 4].values.reshape(-1, 1)  # values converts it into a numpy array ->lnPM2.5
Y = df_final.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column-> Death
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions
plt.plot(X, Y_pred, color='red')

plt.show()

### d) Deaths vs lnNO2 

In [ ]:
df_final.plot.scatter(x='ln_no2',
                     y='death',
                     ylim=(-10,600))
# Add title and axis names
plt.title('Deaths vs lnNO2 scatter chart')
plt.xlabel('lnNO2')
plt.ylabel('Deaths')

#Create linear regression
X = df_final.iloc[:, 5].values.reshape(-1, 1)  # values converts it into a numpy array -> lnNO2
Y = df_final.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column-> Death
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions
plt.plot(X, Y_pred, color='red')

plt.show()

### e) Deaths vs dPM2.5

In [ ]:
df_final.plot.scatter(x='d_pm25',
                     y='death',
                     ylim=(-10,600))
# Add title and axis names
plt.title('Deaths vs dPM2.5 scatter chart')
plt.xlabel('dPM2.5')
plt.ylabel('Deaths')

#Create linear regression
##small comment: DataFrame has NaN values so we have at least 2 solution: 
                    # 1. remove rows with NaN values
                    # 2. replace the NaN values with the mean -> I chose the second one because it seems more reasonable to me
                        ##if you have other suggestions tell me
            
X_new=df_final.iloc[:, 7].fillna(df_final.iloc[:, 7].mean()) #replace the NaN values with the mean 
X = X_new.values.reshape(-1, 1)  # values converts it into a numpy array -> dPM2.5
Y = df_final.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column-> Death
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions
plt.plot(X, Y_pred, color='red')

plt.show()

### f) Deaths vs dNO2

In [ ]:
df_final.plot.scatter(x='d_no2',
                     y='death',
                     ylim=(-10,600)
                     )
# Add title and axis names
plt.title('Deaths vs dNO2 scatter chart')
plt.xlabel('dNO2')
plt.ylabel('Deaths')

#Create linear regression
X_new=df_final.iloc[:, 8].fillna(df_final.iloc[:, 8].mean()) #replace the NaN values with the mean
X = X_new.values.reshape(-1, 1)  # values converts it into a numpy array -> dNO2
Y = df_final.iloc[:, 3].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column-> Death
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions
plt.plot(X, Y_pred, color='red')

plt.show()

In [ ]:
# 3.5.2) Plotting by date